In [65]:
import pandas as pd 
df = pd.read_csv('craft_finalv2.csv')
df.head()

,question,context,steps,final_ans
0,What is the primary focus of gross anatomy in ...,The chunk to be situated within the whole docu...,"['To answer the question, we need to understan...",The primary focus of gross anatomy in the fiel...
1,What is the term for the study of cells and ti...,The chunk to be situated within the whole docu...,"['To answer the question, we need to understan...",histology
2,What is the role of anatomy in the practice of...,The chunk to be situated within the whole docu...,['##begin_quote##The term anatomy tends to mea...,Anatomy forms the basis for the practice of me...
3,What are the two primary techniques a student ...,To situate the provided chunk within the overa...,"['To answer the question, we need to identify ...",observation and visualization
4,How is anatomy more than just memorization of ...,To situate the provided chunk within the overa...,['##begin_quote## Anatomy is much more than ju...,Gross anatomy should be studied using observat...


In [71]:
idx = df.question.apply(lambda x : 'question' in x.lower())
print(df[idx].question)

2831    What was the patient treated for before being ...
Name: question, dtype: object


In [67]:
df.iloc[2831,0]

'What was the patient treated for before being recalled for further questioning?'

In [68]:
idx = df.final_ans.apply(lambda x : '<answer>' in x.lower())
df[idx].final_ans

Series([], Name: final_ans, dtype: object)

In [69]:
# s = df.final_ans.str.replace('<ANSWER>:', '', regex=False)
# df.final_ans = s

In [70]:
df.final_ans.apply(lambda x : '<answer>' in x.lower()).sum()

0

In [76]:
distinct_df = df[df.context.duplicated(keep='last').apply(lambda item: not item)]
distinct_df.shape

(3017, 4)

In [73]:
test_idx = []
for idx , i in enumerate(distinct_df.index.to_list()):
    if idx == 54:
        break
    test_idx.append(i)
print(len(test_idx))
print(test_idx[0],test_idx[-1])

54
2 161


In [74]:
dev_idx = []
for idx , i in enumerate(distinct_df.index.to_list()[54:]):
    if idx == 1800:
        break
    dev_idx.append(i)
print(len(dev_idx))
print(dev_idx[0],dev_idx[-1])

1800
164 5561


In [78]:
llm_df = df.iloc[:,[0,3]]
llm_df.head(),\
llm_df.shape

(                                            question  \
 0  What is the primary focus of gross anatomy in ...   
 1  What is the term for the study of cells and ti...   
 2  What is the role of anatomy in the practice of...   
 3  What are the two primary techniques a student ...   
 4  How is anatomy more than just memorization of ...   
 
                                            final_ans  
 0  The primary focus of gross anatomy in the fiel...  
 1                                          histology  
 2  Anatomy forms the basis for the practice of me...  
 3                      observation and visualization  
 4  Gross anatomy should be studied using observat...  ,
 (9054, 2))

In [84]:
llm_df = llm_df.rename(columns={"question": "questions", "final_ans": "answer"})

In [86]:
test_set = llm_df.iloc[test_idx,:]
test_set.head()

,questions,answer
2,What is the role of anatomy in the practice of...,Anatomy forms the basis for the practice of me...
5,What is the key difference between knowing the...,Knowing the names of the branches of the exter...
8,What is a learning aid that has replaced disse...,"viewing prosected material and plastic models,..."
11,What is the primary focus of the cardiovascula...,The heart and all of the blood vessels in the...
14,What is a limitation of the systemic approach ...,The systemic approach has a limitation in coor...


In [87]:
dev_set = llm_df.iloc[dev_idx,:]
dev_set.head()

,questions,answer
164,What type of muscle is found in the walls of b...,Smooth muscle
167,What is the function of the tunica intima in t...,The tunica intima is the inner endothelial lin...
170,What is the primary function of the tunica med...,To regulate their diameter and control the flo...
173,Which type of veins contain small amounts of s...,Small and medium veins
176,Where do lymphatic vessels ultimately connect ...,large veins in the root of the neck


In [96]:
all_idx = []
train_idx = []
all_idx.extend(test_idx)
all_idx.extend(dev_idx)
for idx , i in enumerate(llm_df.index.to_list()):
    if i not in all_idx:
        train_idx.append(i)
len(train_idx)

7200

In [98]:
164 in train_idx

False

In [99]:
train_set = llm_df.iloc[train_idx,:]
train_set.head()

,questions,answer
0,What is the primary focus of gross anatomy in ...,The primary focus of gross anatomy in the fiel...
1,What is the term for the study of cells and ti...,histology
3,What are the two primary techniques a student ...,observation and visualization
4,How is anatomy more than just memorization of ...,Gross anatomy should be studied using observat...
6,What is the Greek word that is the root of the...,temnein


In [100]:
pd.DataFrame(data={"index": test_idx}).to_csv("./test_idx.csv", sep=',',index=False)
pd.DataFrame(data={"index": dev_idx}).to_csv("./dev_idx.csv", sep=',',index=False)
pd.DataFrame(data={"index": train_idx}).to_csv("./train_idx.csv", sep=',',index=False)


In [101]:
test_set.to_csv("../training_llm/test_set.csv",index=False)
dev_set.to_csv("../training_llm/dev_set.csv",index=False)
train_set.to_csv("../training_llm/train_set.csv",index=False)


In [10]:
import pandas as pd 
train_idx = pd.read_csv('./train_idx.csv')
dev_idx = pd.read_csv('./dev_idx.csv')
test_idx = pd.read_csv('./test_idx.csv')

In [12]:
train_idx_ls = train_idx['index'].to_list()
dev_idx_ls = dev_idx['index'].to_list()
test_idx_ls = test_idx['index'].to_list()

In [17]:
data = pd.read_csv('Dataset_craft.csv')
train_set = data.iloc[train_idx_ls,:]
dev_set = data.iloc[dev_idx_ls,:]
test_set = data.iloc[test_idx_ls,:]



In [21]:
test_set.to_csv("../training_llm/craft_test_set.csv",index=False)
dev_set.to_csv("../training_llm/craft_dev_set.csv",index=False)
train_set.to_csv("../training_llm/craft_train_set.csv",index=False)

In [18]:
train_set

,instruction,output
0,Question: What is the primary focus of gross a...,"<REASON>To answer the question, we need to und..."
1,Question: What is the term for the study of ce...,"<REASON>To answer the question, we need to und..."
3,Question: What are the two primary techniques ...,"<REASON>To answer the question, we need to ide..."
4,Question: How is anatomy more than just memori...,<REASON>##begin_quote## Anatomy is much more t...
6,Question: What is the Greek word that is the r...,<REASON>The question is asking for the Greek w...
...,...,...
9049,Question: What is the order of the structures ...,"<REASON>First, identify the starting point of ..."
9050,Question: What is the name of the tract that c...,"<REASON>First, we need to identify the part of..."
9051,Question: What is the main pathway described b...,"<REASON>""The question asks for the main pathwa..."
9052,Question: What is the function of the cingulum...,<REASON>##begin_quote##The cingulum fibers fro...


In [19]:
dev_set

,instruction,output
164,Question: What type of muscle is found in the ...,<REASON>Identify the type of muscle found in t...
167,Question: What is the function of the tunica i...,<REASON>##begin_quote##The cardiovascular syst...
170,Question: What is the primary function of the ...,<REASON>The question asks about the primary fu...
173,Question: Which type of veins contain small am...,<REASON>Identify the question and the relevant...
176,Question: Where do lymphatic vessels ultimatel...,<REASON>Lymphatic vessels are described as an ...
...,...,...
5549,Question: How many muscles are in the superfic...,<REASON>The question asks how many muscles are...
5552,Question: What nerve innervates the brachiorad...,<REASON>The brachioradialis muscle is located ...
5555,Question: What nerve innervates the extensor c...,<REASON>Read the context to identify the nerve...
5558,Question: From where does the extensor digitor...,<REASON>The question is asking for the origin ...


In [20]:
test_set

,instruction,output
2,Question: What is the role of anatomy in the p...,<REASON>##begin_quote##The term anatomy tends ...
5,Question: What is the key difference between k...,<REASON>##begin_quote##Knowing the names of th...
8,Question: What is a learning aid that has repl...,<REASON>The context mentions that the study of...
11,Question: What is the primary focus of the car...,<REASON>##begin_quote##The regional approach i...
14,Question: What is a limitation of the systemic...,<REASON>##begin_quote##The regional approach w...
17,Question: What is the facial expression in the...,<REASON>The anatomical position is the standar...
20,Question: What is the term for the plane that ...,<REASON>The question asks for the term for the...
23,Question: What is the relationship between the...,<REASON>The first step is to understand the co...
26,Question: What is the term 'cranial' used to d...,"<REASON>""The question asks for the term 'crani..."
29,Question: Which structure is superficial to th...,"<REASON>To answer the question, we need to und..."
